In [ ]:
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()

# Get OPENAI_API_KEY from .env file
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ["OPENAI_ORGANIZATION"] = os.getenv('OPENAI_ORGANIZATION')

### Set up the vector database

In [ ]:
from weaviate import Client

# Initialize Weaviate client
client = Client("http://localhost:8080")

In [ ]:
def create_schema():
    """
    Creates the schema for the 'code_example' class in Weaviate.
    """
    class_obj = {
        "class": "code_example",
        "vectorizer": "text2vec-openai",
        "moduleConfig": {
            "text2vec-openai": {}
        },
        "properties": [
            {
                "name": "file_name",
                "dataType": ["string"],
                "description": "File name associated with the code"
            },
            {
                "name": "tags",
                "dataType": ["string[]"],
                "description": "Tags associated with the code"
            },
            {
                "name": "description",
                "dataType": ["text"],
                "description": "Description of the code",
                "moduleConfig": {
                    "text2vec-openai": {
                        "vectorizePropertyName": 'true'
                    }
                },
            },
            {
                "name": "code",
                "dataType": ["text"],
                "description": "The actual code"
            }
        ]
    }

    client.schema.delete_all()  # Clear existing schema
    client.schema.create_class(class_obj)  # Create the new class

# Create the schema
create_schema()

### Populate the weaviate database

In [ ]:
from weaviate.util import generate_uuid5 

def populate_weaviate(documents):
    """
    Populates the Weaviate database with the provided documents.
    Clears the database if it is already populated.
    """

    # Configure batch
    client.batch.configure(batch_size=len(documents))

    # Initialize a batch process
    with client.batch as batch:
        # Batch import data
        for i, doc in enumerate(documents):
            print(f"Importing document: {i+1}")
            try:
                properties = {
                    "file_name": doc["file_name"],
                    "tags": doc["tags"],
                    "description": doc["description"],
                    "code": doc["code"]
                }
                batch.add_data_object(
                    data_object=properties,
                    class_name="code_example",
                    uuid=generate_uuid5(properties["file_name"])
                )
            except Exception as e:
                print(f"Error adding document to Weaviate: {e}")

In [ ]:
import os
import json
import yaml

# Define the path to the data directory
DATA_DIR = "../data"

# Define a function to extract tags, description, and code from a file
def extract_information(file_path):
    with open(file_path, 'r') as file:
        content = file.read()

    # Extract the docstring
    docstring_start = content.find('"""') + 3
    docstring_end = content.find('"""', docstring_start)
    docstring = content[docstring_start:docstring_end]

    # Parse the docstring as YAML
    metadata = yaml.safe_load(docstring)

    # Extract tags and description
    tags = metadata['tags']
    description = metadata['description']

    # Extract code
    code_start = content.find('"""', docstring_end) + len('"""')
    if code_start != -1:
        code = content[code_start:].strip()
    else:
        print(f"Error in file {file_path}: No code found after docstring.")
        return

    return {
        'file_name': os.path.basename(file_path),
        'tags': tags,
        'description': description,
        'code': code
    }

In [ ]:
def process_files():
    """
    Modified function to process files and populate Weaviate database.
    """
    documents = []
    for file_name in os.listdir(DATA_DIR):
        if file_name.endswith('.py'):
            file_path = os.path.join(DATA_DIR, file_name)
            data = extract_information(file_path)
            
            if data:
                documents.append({"file_name": file_name, **data})
            else:
                log_error(file_name, "tags, description, or code")

    # Populate Weaviate with extracted documents
    populate_weaviate(documents)

In [ ]:
# Run the main function
process_files()

### Query the database

In [ ]:
def query_collection(query):
    response = (
        client.query
        .get("code_example", ["description", "code", "file_name", "tags"])
        .with_near_text({
            "concepts": [query]
        })
        .with_limit(3)
        .with_additional(["distance"])
        .do()
    )
    return response

In [ ]:
def format_query_results(query_result):
    for result in query_result['data']['Get']['Code_example']:
        print("# Example:")
        print(f"{result['code']}\n")

In [ ]:
new_request = """
- Accept string
- Retrieve matching documents using Weaviate vectorstore
- Format single prompt with output instructions
- Parse response as string
"""

In [ ]:
# Query the collection
query_result = query_collection(new_request)

# Format and print the results
format_query_results(query_result)